In [ ]:
import os
import re
import time
from tqdm import tqdm


LOG_PATH = "/net/scratch/people/plgjacekh/slurm-log/"

In [ ]:
def spark_progress_bar(log, refresh=5):
    spark_pbar_re = re.compile(
        r'.*\[Stage (?P<stage>\d+):=*>?' +
        r'\s*\((?P<elapsed>\d+) \+ (?P<cpus>\d+)\) / (?P<total>\d+)\]'
    )

    while True:
        with open(log) as f:
            log_lines = f.readlines()
        match = spark_pbar_re.match(log_lines[-1])
        if match is not None:
            last_stage = match.group('stage')
            last_elapsed = int(match.group('elapsed'))
            total = int(match.group('total'))
            break
        else:
            time.sleep(refresh)

    pbar = tqdm(total=total)
    pbar.set_description(f'Stage {last_stage}')
    pbar.update(last_elapsed)
    try:
        while True:
            time.sleep(refresh)
            with open(log, 'rt') as f:
                log_lines = f.readlines()
            match = spark_pbar_re.match(log_lines[-1])
            elapsed = int(match.group('elapsed'))
            stage = match.group('stage')
            if stage == last_stage:
                pbar.update(elapsed - last_elapsed)
            else:
                pbar.close()
                total = int(match.group('total'))
                pbar = tqdm(total=total)
                pbar.set_description(f'Stage {stage}')
                pbar.update(elapsed)
                last_stage = stage
            last_elapsed = elapsed
    except KeyboardInterrupt:
        pbar.close()

In [ ]:
job_id = '10355407'
spark_progress_bar(
    log=f"{LOG_PATH}/{job_id}.err",
    refresh=10
)

In [ ]:
spark_progress_bar(
    log=f"{LOG_PATH}/jupyter-{os.environ.get('SLURM_JOB_ID')}.err",
    refresh=10
)